In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer # UPDATED: Import Aer from the new package
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def deutsch_algorithm(oracle_type='balanced'):
    """
    Implements Deutsch's Algorithm based on the provided derivation.
    """
    # Initialize a quantum circuit with 2 qubits and 1 classical bit
    qc = QuantumCircuit(2, 1)
    
    # --- Step 0: Initialization ---
    # The text specifies the initial state as |psi_0> = |0>|1>.
    # Qiskit initializes qubits to |0>, so we apply an X gate to Qubit 1 to make it |1>.
    qc.x(1) 
    qc.barrier()
    
    # --- Step 1: Apply Hadamard Gates ---
    qc.h(0)
    qc.h(1)
    qc.barrier()
    
    # --- Step 2: Apply the Oracle U_f ---
    if oracle_type == 'balanced':
        # Balanced: f(x) = x (CNOT acts as x XOR y)
        qc.cx(0, 1) 
    elif oracle_type == 'constant':
        # Constant: f(x) = 0 (Identity, do nothing)
        pass 
        
    qc.barrier()
    
    # --- Step 3: Apply Final Hadamard ---
    qc.h(0)
    qc.barrier()
    
    # --- Step 4: Measurement ---
    qc.measure(0, 0)
    
    return qc

# Create the circuit
qc = deutsch_algorithm(oracle_type='balanced')

# Draw the circuit diagram
print("Circuit Diagram:")
print(qc.draw(output='text'))

# --- Simulation ---
# UPDATED: Use the Aer simulator from the correct package
simulator = Aer.get_backend('aer_simulator')

# Transpile the circuit for the simulator
transpiled_qc = transpile(qc, simulator)

# Run the simulation
result = simulator.run(transpiled_qc).result()
counts = result.get_counts()

print("\nMeasurement Result (0 = Constant, 1 = Balanced):")
print(counts)

Circuit Diagram:
           ░ ┌───┐ ░       ░ ┌───┐ ░ ┌─┐
q_0: ──────░─┤ H ├─░───■───░─┤ H ├─░─┤M├
     ┌───┐ ░ ├───┤ ░ ┌─┴─┐ ░ └───┘ ░ └╥┘
q_1: ┤ X ├─░─┤ H ├─░─┤ X ├─░───────░──╫─
     └───┘ ░ └───┘ ░ └───┘ ░       ░  ║ 
c: 1/═════════════════════════════════╩═
                                      0 

Measurement Result (0 = Constant, 1 = Balanced):
{'1': 1024}
